<a href="https://colab.research.google.com/github/M-Abbi/Financial-Modeling/blob/main/ARMA_%2B_GARCH_Simple_Trading_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import yfinance as yf
import pandas as pd
import numpy as np
from arch import arch_model
import matplotlib.pyplot as plt
import warnings

In [6]:
warnings.filterwarnings('ignore') # Suppress convergence warnings for cleaner output

In [7]:
## Fetching daily adjusted close prices and calculating percentage returns.
# --- 1. Data Fetching ---
def get_stock_data(ticker, start_date, end_date):
    """Fetches stock data from Yahoo Finance."""
    data = yf.download(ticker, start=start_date, end=end_date, progress=False)
    data['Returns'] = data['Adj Close'].pct_change() * 100 # Percentage returns
    data.dropna(inplace=True)
    return data

In [8]:
# --- 2. GARCH Model and Forecasting ---
def garch_forecast(returns_series):
    """
    Fits an ARMA(1,1)-GARCH(1,1) model and forecasts one step ahead.
    Returns: (predicted_mean_return, predicted_volatility)
    """
    try:
        # p=1, q=1 for GARCH; lags=1 for AR(1) in mean model.
        # You can use 'Constant' for mean if you only want GARCH on residuals of constant mean.
        # Using AR(1) for a very simple attempt at mean forecasting.
        # dist='Normal' assumes normally distributed errors.
        model = arch_model(returns_series, vol='Garch', p=1, q=1, mean='ARX', lags=1, dist='Normal')
        res = model.fit(disp='off', update_freq=0) # disp='off' to suppress output

        # res.forecast(horizon=1) predicts one step ahead
        forecast = res.forecast(horizon=1)
        # Returns the predicted mean return (predicted_mu) and predicted standard deviation (predicted_vol)
        predicted_mu = forecast.mean['h.1'].iloc[-1]
        predicted_vol = np.sqrt(forecast.variance['h.1'].iloc[-1])
        return predicted_mu, predicted_vol

    # Includes a try-except block because GARCH models can sometimes fail to converge, especially with small windows or certain data patterns.

    except Exception as e:
        # print(f"Warning: GARCH model fitting failed: {e}")
        return np.nan, np.nan # Return NaN if model fails

In [9]:
# Iterates through the data, day by day, after an initial window_size period.
# --- 3. Backtesting Engine ---
def backtest_strategy(data, window_size=100, entry_threshold_std=0.5, exit_threshold_std=0.0):
    """
    Performs a backtest of the GARCH-based trading strategy.
    entry_threshold_std: How many std devs above zero the mean forecast must be to buy.
    exit_threshold_std: If mean forecast drops below this * std dev, sell.
    """
    signals = pd.DataFrame(index=data.index)
    signals['Price'] = data['Adj Close']
    signals['Returns'] = data['Returns']
    signals['Signal'] = 0 # 0: Hold, 1: Buy, -1: Sell
    signals['Position'] = 0 # 0: No position, 1: Long
    signals['Predicted_Mean'] = np.nan
    signals['Predicted_Vol'] = np.nan

    cash = 100000  # Initial cash
    portfolio_value = [cash]
    position_active = False
    entry_price = 0
    trades_log = [] # To store trade details

    for i in range(window_size, len(data)):
        current_date = data.index[i]
        current_price = data['Adj Close'].iloc[i]

        # Get data for GARCH model
        historical_returns = data['Returns'].iloc[i-window_size:i]

        if len(historical_returns) < 20: # Need enough data points
            signals['Position'].iloc[i] = signals['Position'].iloc[i-1]
            portfolio_value.append(portfolio_value[-1]) # No change if no position
            continue

        pred_mu, pred_vol = garch_forecast(historical_returns)
        signals['Predicted_Mean'].iloc[i] = pred_mu
        signals['Predicted_Vol'].iloc[i] = pred_vol

        # --- Trading Logic ---
        if not position_active:
            # Buy condition - If not in a position and pred_mu > entry_threshold_std * pred_vol, a BUY signal is generated.
            if not np.isnan(pred_mu) and not np.isnan(pred_vol) and pred_vol > 0: # Check for valid forecasts
                if pred_mu > entry_threshold_std * pred_vol:
                    signals['Signal'].iloc[i] = 1
                    signals['Position'].iloc[i] = 1
                    position_active = True
                    entry_price = current_price
                    # Assuming we buy one share for simplicity, adjust cash
                    # In a real system, you'd calculate shares based on cash and price
                    trades_log.append({'Date': current_date, 'Type': 'BUY', 'Price': current_price, 'Pred_Mu': pred_mu, 'Pred_Vol': pred_vol})
                    # print(f"{current_date}: BUY signal at {current_price:.2f} (Pred Mu: {pred_mu:.2f}, Pred Vol: {pred_vol:.2f})")
        else: # Position is active, look for sell signal
            signals['Position'].iloc[i] = 1 # Carry forward position

            # Sell condition - If in a position and pred_mu < exit_threshold_std * pred_vol, a SELL signal is generated.
            # Example: Sell if predicted mean return turns negative or small positive
            if not np.isnan(pred_mu) and not np.isnan(pred_vol) and pred_vol > 0:
                if pred_mu < exit_threshold_std * pred_vol:
                    signals['Signal'].iloc[i] = -1
                    signals['Position'].iloc[i] = 0
                    position_active = False
                    exit_price = current_price
                    trade_return = (exit_price - entry_price) / entry_price
                    trades_log.append({'Date': current_date, 'Type': 'SELL', 'Price': current_price, 'Return': trade_return, 'Pred_Mu': pred_mu, 'Pred_Vol': pred_vol})
                    # print(f"{current_date}: SELL signal at {exit_price:.2f} (Return: {trade_return*100:.2f}%)")
                    entry_price = 0 # Reset entry price

        # Update portfolio value (simplified: assumes holding 1 share or cash)
        # For a more robust backtest, you'd track shares and cash properly.
        # This part is a bit wavy for simplicity as it doesn't track shares properly
        # A proper portfolio valuation:
        if i > 0: # Ensure we have a previous value
            daily_return_from_asset = 0
            if signals['Position'].iloc[i-1] == 1: # If we held the asset yesterday
                daily_return_from_asset = data['Returns'].iloc[i]/100 # Use actual daily return

            # This is still simplified as it assumes full investment or full cash
            # A better way to be implemented: track cash and number of shares
            if signals['Position'].iloc[i-1] == 1:
                 portfolio_value.append(portfolio_value[-1] * (1 + daily_return_from_asset))
            else:
                 portfolio_value.append(portfolio_value[-1]) # Just cash, no change

    # Ensure portfolio_value has the same length as signals for plotting
    # Pad beginning of portfolio_value if necessary (for window_size period)
    initial_portfolio_values = [cash] * window_size
    full_portfolio_value = initial_portfolio_values + portfolio_value[1:] # portfolio_value[0] is initial cash

    # Make sure it aligns with signals index
    if len(full_portfolio_value) > len(signals):
        full_portfolio_value = full_portfolio_value[:len(signals)]
    elif len(full_portfolio_value) < len(signals):
        full_portfolio_value.extend([full_portfolio_value[-1]]*(len(signals) - len(full_portfolio_value)))

    signals['Portfolio_Value'] = pd.Series(full_portfolio_value, index=signals.index)

    return signals, pd.DataFrame(trades_log)


In [10]:
# Calculates Total Return, Annualized Sharpe Ratio (assuming 252 trading days), and Max Drawdown for the strategy.

# --- 4. Performance Metrics ---
def calculate_performance(signals_df, initial_capital):
    """Calculates and prints performance metrics."""
    portfolio_value = signals_df['Portfolio_Value']

    if portfolio_value.empty or portfolio_value.iloc[-1] == 0:
        print("Portfolio value is zero or empty, cannot calculate metrics.")
        return

    total_return = (portfolio_value.iloc[-1] - initial_capital) / initial_capital

    # Daily returns of the strategy
    strategy_returns = portfolio_value.pct_change().dropna()

    if not strategy_returns.empty:
        sharpe_ratio_annualized = (np.mean(strategy_returns) / np.std(strategy_returns)) * np.sqrt(252) if np.std(strategy_returns) != 0 else 0
    else:
        sharpe_ratio_annualized = 0

    # Max Drawdown
    cumulative_returns = (1 + strategy_returns).cumprod()
    peak = cumulative_returns.cummax()
    drawdown = (cumulative_returns - peak) / peak
    max_drawdown = drawdown.min()

    print(f"\n--- Strategy Performance ---")
    print(f"Initial Capital: ${initial_capital:,.2f}")
    print(f"Final Portfolio Value: ${portfolio_value.iloc[-1]:,.2f}")
    print(f"Total Return: {total_return*100:.2f}%")
    print(f"Annualized Sharpe Ratio: {sharpe_ratio_annualized:.2f}")
    print(f"Max Drawdown: {max_drawdown*100:.2f}%")

    # Buy and Hold performance - shows Buy & Hold return for comparison.
    buy_hold_return = (signals_df['Price'].iloc[-1] - signals_df['Price'].iloc[0]) / signals_df['Price'].iloc[0]
    print(f"\n--- Buy & Hold Performance ---")
    print(f"Buy & Hold Total Return: {buy_hold_return*100:.2f}%")

In [14]:
# --- 5. Plotting ---
def plot_results(signals_df, ticker):
    """Plots the price, signals, and portfolio value."""
    fig, ax1 = plt.subplots(figsize=(14, 7))

    # Price
    color = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price', color=color)
    ax1.plot(signals_df.index, signals_df['Price'], color=color, label='Stock Price')
    ax1.tick_params(axis='y', labelcolor=color)

    # Buy/Sell signals on price chart
    ax1.plot(signals_df[signals_df['Signal'] == 1].index,
             signals_df['Price'][signals_df['Signal'] == 1],
             '^', markersize=10, color='g', lw=0, label='Buy Signal')
    ax1.plot(signals_df[signals_df['Signal'] == -1].index,
             signals_df['Price'][signals_df['Signal'] == -1],
             'v', markersize=10, color='r', lw=0, label='Sell Signal')
    ax1.legend(loc='upper left')
    ax1.set_title(f'{ticker} Trading Strategy Backtest')

    # Portfolio Value on a second y-axis
    ax2 = ax1.twinx()
    color = 'tab:red'
    ax2.set_ylabel('Portfolio Value', color=color)
    ax2.plot(signals_df.index, signals_df['Portfolio_Value'], color=color, linestyle='--', label='Strategy Value')

    # Buy & Hold Equity Curve
    initial_capital = signals_df['Portfolio_Value'].iloc[0]
    buy_hold_equity = initial_capital * (1 + signals_df['Price'].pct_change().fillna(0).cumsum())
    ax2.plot(signals_df.index, buy_hold_equity, color='tab:purple', linestyle=':', label='Buy & Hold Value')

    ax2.tick_params(axis='y', labelcolor=color)
    ax2.legend(loc='center left')

    fig.tight_layout()
    plt.show()

    # Plot predicted mean and volatility for insight (optional)
    fig, (ax_mu, ax_vol) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
    ax_mu.plot(signals_df.index, signals_df['Predicted_Mean'], label='Predicted Mean Return', color='orange')
    ax_mu.axhline(0, color='gray', linestyle='--')
    ax_mu.set_ylabel('Predicted Mean (%)')
    ax_mu.legend()
    ax_mu.set_title('ARMA-GARCH Forecasts')

    ax_vol.plot(signals_df.index, signals_df['Predicted_Vol'], label='Predicted Volatility (Std Dev)', color='green')
    ax_vol.set_ylabel('Predicted Volatility (%)')
    ax_vol.legend()
    plt.show()

In [16]:
# --- Main Execution ---
if __name__ == "__main__":
    TICKER = 'RACE'  # Example: Apple
    START_DATE = '2021-01-01'
    END_DATE = '2025-05-01'
    ROLLING_WINDOW = 100       # Days for GARCH model fitting
    ENTRY_STD_THRESHOLD = 0.5 # Buy if pred_mu > 0.5 * pred_vol
    EXIT_STD_THRESHOLD = 0.0  # Sell if pred_mu < 0.0 * pred_vol (i.e., turns negative)
    INITIAL_CAPITAL = 100000

    stock_data = get_stock_data(TICKER, START_DATE, END_DATE)

    if not stock_data.empty:
        print(f"Running backtest for {TICKER}...")
        backtest_signals, trades = backtest_strategy(stock_data,
                                                     window_size=ROLLING_WINDOW,
                                                     entry_threshold_std=ENTRY_STD_THRESHOLD,
                                                     exit_threshold_std=EXIT_STD_THRESHOLD)

        print("\n--- Trades Log ---")
        if not trades.empty:
            print(trades.to_string())
            # Calculate trade-specific metrics
            winning_trades = trades[trades['Return'] > 0] if 'Return' in trades.columns else pd.DataFrame()
            losing_trades = trades[trades['Return'] <= 0] if 'Return' in trades.columns else pd.DataFrame()
            num_trades = len(trades[trades['Type'] == 'SELL']) # Count closed trades

            if num_trades > 0:
                win_rate = len(winning_trades) / num_trades if num_trades > 0 else 0
                avg_win_return = winning_trades['Return'].mean() if not winning_trades.empty else 0
                avg_loss_return = losing_trades['Return'].mean() if not losing_trades.empty else 0
                print(f"\nNumber of Trades (closed): {num_trades}")
                print(f"Win Rate: {win_rate*100:.2f}%")
                print(f"Average Win Return: {avg_win_return*100:.2f}%")
                print(f"Average Loss Return: {avg_loss_return*100:.2f}%")

        else:
            print("No trades were executed.")

        calculate_performance(backtest_signals, initial_capital=INITIAL_CAPITAL) # Use initial capital for calc
        plot_results(backtest_signals, TICKER)
    else:
        print(f"No data found for {TICKER} in the given date range.")

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['RACE']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


No data found for RACE in the given date range.
